In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import shuffle

# Load your dataset
df = pd.read_csv('PAT_Dataset.csv')

# Shuffle the dataset
df = shuffle(df, random_state=42)

# Columns containing the target variables
target_columns = [
    'steady_state_temp_L0', 'steady_state_temp_L1', 'router_avg_temp_L0', 'router_avg_temp_L1',
    'core_avg_temp_L0', 'core_avg_temp_L1', 'mem_avg_temp_L0', 'mem_avg_temp_L1',
    'total_area', 'avg_power', 'avg_cores_power', 'avg_routers_power', 'avg_power_per_router',
    'layer_area', 'area_per_core'
]

# Identify categorical columns
categorical_columns = ['routing_type', 'selection_strategy', 'traffic_type']

# One-hot encode categorical columns
X = pd.get_dummies(df.drop(columns=target_columns), columns=categorical_columns, drop_first=True)
y = df[target_columns]

# Split the data into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize StandardScaler
scaler_X = StandardScaler()
scaler_y = StandardScaler()

# Scale the feature and target data
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)


In [4]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dropout
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Reshape the data to fit a CNN input
# The CNN model expects a 3D input shape (samples, time steps, features), so we reshape X_train_scaled and X_test_scaled
X_train_cnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)  # Add 1 for the single "channel"
X_test_cnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

# Build an improved CNN model with dropout layers and deeper architecture
cnn_model = Sequential()

cnn_model.add(Conv1D(filters=64, kernel_size=3, activation='relu', padding='same', input_shape=(X_train_cnn.shape[1], 1)))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Dropout(0.3))

cnn_model.add(Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Dropout(0.3))

cnn_model.add(Conv1D(filters=256, kernel_size=3, activation='relu', padding='same'))
cnn_model.add(MaxPooling1D(pool_size=2))
cnn_model.add(Dropout(0.3))

# Flatten and add dense layers for regression
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(Dropout(0.3))
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(len(target_columns)))  # Output layer for all targets at once

# Compile the model
cnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Set up callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)

# Train the model
cnn_model.fit(X_train_cnn, y_train_scaled, epochs=50, batch_size=32, validation_split=0.2,
              callbacks=[early_stopping, reduce_lr])

# Predict using the trained CNN model
y_pred_cnn_scaled = cnn_model.predict(X_test_cnn)

# Prepare evaluation results as before
cnn_evaluation_results = []
print("CNN Performance (per target variable):")

for i, target in enumerate(target_columns):
    y_test_target = y_test_scaled[:, i]
    y_pred_target = y_pred_cnn_scaled[:, i]
    mse = mean_squared_error(y_test_target, y_pred_target)
    mae = mean_absolute_error(y_test_target, y_pred_target)
    r2 = r2_score(y_test_target, y_pred_target)
    cnn_evaluation_results.append({
        'algorithm': 'CNN',
        'target': target,
        'MSE': mse,
        'MAE': mae,
        'R²': r2
    })
    print(f"\nTarget: {target}")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R²:", r2)
    print("-" * 30)

# Results DataFrame
cnn_evaluation_df = pd.DataFrame(cnn_evaluation_results)
print("\nCNN Evaluation Results DataFrame:")
print(cnn_evaluation_df)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - loss: 0.5592 - val_loss: 0.1748 - learning_rate: 0.0010
Epoch 2/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.2413 - val_loss: 0.0981 - learning_rate: 0.0010
Epoch 3/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.1725 - val_loss: 0.0930 - learning_rate: 0.0010
Epoch 4/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.1461 - val_loss: 0.0641 - learning_rate: 0.0010
Epoch 5/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 0.1270 - val_loss: 0.0554 - learning_rate: 0.0010
Epoch 6/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.1149 - val_loss: 0.0479 - learning_rate: 0.0010
Epoch 7/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - loss: 0.1096 - val_loss: 0.0532 - learning_rate: 0.0010
Epoch 8/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - loss: 0.1001 - val_loss: 0.0492 - learning_rate: 0.0010
Epoch 9/50
516/516 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - loss: 0.0931 - val_loss: 0.0395 - learning_rate: 0.0010
Epoch 

In [17]:
import pandas as pd

new_data = pd.DataFrame({
    'dimx': [75],
    'dimy': [75],
    'dimz': [2],
    'buffer_size': [10],
    'packet_size_min': [4],
    'packet_size_max': [8],
    'routing_type_xyz': [1],  # Make sure to align with the encoded columns
    'selection_strategy_thermal': [1],
    'traffic_type_random': [1],
    'injection_rate': [0.05]
})

# Ensure all columns are present (including any missing dummy variables from the training data)
# Missing columns in new_data are filled with 0s
for col in X.columns:
    if col not in new_data:
        new_data[col] = 0

# Reorder the new_data columns to match the original feature order
new_data = new_data[X.columns]

# Scale the new data using the previously fitted scaler
new_data_scaled = scaler_X.transform(new_data)

# Reshape the scaled data to fit the CNN input shape
new_data_cnn = new_data_scaled.reshape(1, new_data_scaled.shape[1], 1)  # Shape (1, num_features, 1)

# Make predictions with the CNN model
new_data_pred_scaled = cnn_model.predict(new_data_cnn)

# Inverse scale the predictions to get original scale
new_data_pred = scaler_y.inverse_transform(new_data_pred_scaled)

# Convert the prediction to a DataFrame for readability
predictions_df = pd.DataFrame(new_data_pred, columns=target_columns)

# Display the predictions
print("Predictions for the new data:")
print(predictions_df)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Predictions for the new data:
   steady_state_temp_L0  steady_state_temp_L1  router_avg_temp_L0  \
0            320.988098            239.610153           31.341198   

   router_avg_temp_L1  core_avg_temp_L0  core_avg_temp_L1  mem_avg_temp_L0  \
0           28.574923         26.519922         26.969509        26.046755   

   mem_avg_temp_L1    total_area  avg_power  avg_cores_power  \
0        26.257633  8.420159e+09   0.000006         0.000004   

   avg_routers_power  avg_power_per_router    layer_area  area_per_core  
0           0.000001          1.240144e-09  4.124060e+09      4695230.0  
